<a href="https://colab.research.google.com/github/HenryZumaeta/MDS_UNI/blob/Zeta/CICLO02/DL/C06_20240515_Guardado_y_carga_de_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guardar y cargar modelos

En este tutorial aprenderemos cómo podemos tomar un modelo entrenado, guardarlo y luego volver a cargarlo para seguir entrenándolo o usarlo para realizar inferencias. En particular, utilizaremos el aprendizaje por transferencia para entrenar un clasificador para clasificar imágenes de perros y gatos, tal como lo hicimos en la lección anterior. Luego tomaremos nuestro modelo entrenado y lo guardaremos como un archivo HDF5, que es el formato utilizado por Keras. Luego cargaremos este modelo, lo usaremos para realizar predicciones y luego continuaremos entrenando el modelo. Finalmente, guardaremos nuestro modelo entrenado como TensorFlow SavedModel y luego lo descargaremos a un disco local, para que luego pueda usarse para su implementación en diferentes plataformas.

## Conceptos que se cubrirán en este Colab

1. Guardar modelos en formato HDF5 para Keras
2. Guardar modelos en el formato TensorFlow SavedModel
3. Cargando modelos
4. Descargar modelos al disco local

Antes de iniciar este Colab, debe restablecer el entorno de Colab seleccionando `Runtime -> Reset all runtimes...` en el menú de arriba.

# Importaciones

En este Colab usaremos la versión Beta de TensorFlow 2.0.

In [ ]:
!pip install -U tensorflow_hub
 # Instala o actualiza la biblioteca TensorFlow Hub para cargar modelos preentrenados desde TensorFlow Hub.

!pip install -U tensorflow_datasets
 # Instala o actualiza la biblioteca TensorFlow Datasets para acceder a conjuntos de datos estándar para tareas de aprendizaje automático.


Algunas importaciones normales que hemos visto antes.

In [ ]:
import time  # Importa el módulo 'time' para trabajar con funciones relacionadas con el tiempo.
import numpy as np  # Importa NumPy, una biblioteca para trabajar con arreglos y operaciones matemáticas.
import matplotlib.pylab as plt  # Importa Matplotlib, una biblioteca para crear gráficos y visualizaciones.

import tensorflow as tf
# Importa TensorFlow, una biblioteca popular para el aprendizaje profundo y otras tareas de aprendizaje automático.
import tensorflow_hub as hub
# Importa TensorFlow Hub para acceder a modelos preentrenados y sus características.
import tensorflow_datasets as tfds
 # Importa TensorFlow Datasets para acceder a conjuntos de datos estándar y facilitar su uso en el aprendizaje automático.
tfds.disable_progress_bar()  # Desactiva la barra de progreso de TensorFlow Datasets para una salida más limpia en la consola.

from tensorflow.keras import layers
# Importa el módulo 'layers' de la biblioteca Keras de TensorFlow para crear capas de redes neuronales y otros componentes del modelo.


# Parte 1: cargar el conjunto de datos de gatos y perros

Usaremos conjuntos de datos de TensorFlow para cargar el conjunto de datos de Perros vs Gatos.

In [ ]:
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',  # Carga el conjunto de datos 'cats_vs_dogs' desde TensorFlow Datasets.
    split=['train[:80%]', 'train[80%:]'],  # Divide el conjunto de entrenamiento en 80% para entrenamiento y 20% para validación.
    with_info=True,  # Incluye información adicional sobre el conjunto de datos, como el número de ejemplos y características.
    as_supervised=True,  # Carga el conjunto de datos en un formato (imagen, etiqueta) para facilitar el uso en tareas supervisadas.
)


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


Las imágenes del conjunto de datos Perros vs. Gatos no son todas del mismo tamaño. Entonces, necesitamos reformatear todas las imágenes a la resolución esperada por MobileNet (224, 224)

In [ ]:
def format_image(image, label):
    # Los módulos de imagen de 'hub' esperan que los datos estén normalizados en el rango [0, 1].
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES)) / 255.0  # Normaliza la imagen al rango [0, 1] y redimensiona a IMAGE_RES x IMAGE_RES.
    return image, label

num_examples = info.splits['train'].num_examples  # Obtiene el número total de ejemplos en el conjunto de entrenamiento.

BATCH_SIZE = 32  # Establece el tamaño del lote para el entrenamiento y la validación.
IMAGE_RES = 224  # Establece la resolución de las imágenes a 224x224 píxeles para el formato utilizado en el modelo.

train_batches = train_examples.cache().shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
# Prepara lotes de entrenamiento: almacena en caché, mezcla, aplica la función de formato de imagen, crea lotes y prefetch para la eficiencia.

validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)
# Prepara lotes de validación: almacena en caché, aplica la función de formato de imagen, crea lotes y prefetch para la eficiencia.


# Parte 2: Transferir el aprendizaje con TensorFlow Hub

Ahora usaremos TensorFlow Hub para realizar Transfer Learning.

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))

Congele las variables en la capa del extractor de características, de modo que el entrenamiento solo modifique la capa del clasificador final.

In [ ]:
feature_extractor.trainable = False
# Establece la capa de extracción de características (MobileNetV2) como no entrenable.
# Al fijar esta capa como no entrenable, sus pesos
# y parámetros no se actualizarán durante el entrenamiento del modelo, lo que preserva las características aprendidas previamente.


## Adjunte un encabezado de clasificación

Ahora envuelva la capa central en un modelo `tf.keras.Sequential` y agregue una nueva capa de clasificación.

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,  # Agrega la capa de extracción de características (MobileNetV2) al modelo secuencial.
    layers.Dense(2)  # Agrega una capa densa con 2 unidades para la clasificación binaria (en este caso, 2 clases: gato y perro).
])

model.summary()
# Muestra un resumen del modelo, incluyendo la arquitectura de las capas,
# el número de parámetros entrenables y no entrenables, y la forma de salida de cada capa.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2260546 (8.62 MB)
Trainable params: 2562 (10.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


## Entrena el modelo

Ahora entrenamos este modelo como cualquier otro, llamando primero a `compile` y luego a `fit`.

In [ ]:
model.compile(
    optimizer='adam',  # Configura el optimizador Adam para el entrenamiento del modelo.
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    # Utiliza la entropía cruzada categórica como función de pérdida para problemas de clasificación.
    metrics=['accuracy']  # Utiliza la precisión como métrica para evaluar el rendimiento del modelo durante el entrenamiento.
)

EPOCHS = 1  # Establece el número de épocas para entrenar el modelo.

history = model.fit(
    train_batches,  # Utiliza el lote de entrenamiento para entrenar el modelo.
    epochs=EPOCHS,  # Número de épocas para entrenar el modelo.
    validation_data=validation_batches
    # Utiliza el lote de validación para evaluar el modelo después de cada época de entrenamiento.
)


582/582 [==============================] - ETA: 0s - loss: 0.0616 - accuracy: 0.9773

## Consulta las predicciones

Obtenga la lista ordenada de nombres de clases.

In [ ]:
class_names = np.array(info.features['label'].names)
# Obtiene los nombres de las clases del conjunto de datos y los almacena en un array NumPy llamado 'class_names'.

print(class_names)
# Imprime los nombres de las clases para mostrar las etiquetas de las clases disponibles en el conjunto de datos.


Ejecute un lote de imágenes a través del modelo y convierta los índices en nombres de clases.

In [ ]:
image_batch, label_batch = next(iter(train_batches.take(1)))
# Obtiene un lote de imágenes y sus etiquetas del conjunto de entrenamiento utilizando la función 'iter' y 'take'.

image_batch = image_batch.numpy()
label_batch = label_batch.numpy()
# Convierte el lote de imágenes y etiquetas de TensorFlow a arrays NumPy para facilitar el procesamiento.

predicted_batch = model.predict(image_batch)
# Realiza predicciones utilizando el modelo en el lote de imágenes obtenido.

predicted_batch = tf.squeeze(predicted_batch).numpy()
# Elimina dimensiones adicionales de las predicciones utilizando 'tf.squeeze' y convierte a un array NumPy.

predicted_ids = np.argmax(predicted_batch, axis=-1)
# Obtiene los índices de las clases predichas con mayor probabilidad para cada imagen.

predicted_class_names = class_names[predicted_ids]
# Obtiene los nombres de las clases predichas utilizando los índices y los nombres de las clases disponibles en 'class_names'.

print(predicted_class_names)
# Imprime los nombres de las clases predichas para el lote de imágenes dado.


Veamos las etiquetas verdaderas y las previstas.

In [ ]:
print("Labels: ", label_batch)
# Imprime las etiquetas reales del lote de imágenes, mostrando las clases verdaderas correspondientes a cada imagen.

print("Predicted labels: ", predicted_ids)
# Imprime las clases predichas para cada imagen en el lote, mostrando los índices de las clases predichas.


In [ ]:
plt.figure(figsize=(10, 9))
for n in range(30):
    plt.subplot(6, 5, n+1)
    plt.imshow(image_batch[n])  # Muestra la imagen en la posición 'n' del lote.

    color = "blue" if predicted_ids[n] == label_batch[n] else "red"
    # Establece el color del título como azul si la predicción es correcta y rojo si es incorrecta.

    plt.title(predicted_class_names[n].title(), color=color)
    # Muestra el nombre de la clase predicha en el título de la imagen, con la primera letra en mayúscula.

    plt.axis('off')  # Desactiva los ejes para mostrar solo las imágenes sin marco.

_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")
# Muestra un título global que indica la interpretación del color en las predicciones (azul: correcto, rojo: incorrecto).


# Parte 3: Guardar como modelo Keras `.h5`

Ahora que hemos entrenado el modelo, podemos guardarlo como un archivo HDF5, que es el formato utilizado por Keras. Nuestro archivo HDF5 tendrá la extensión '.h5' y su nombre corresponderá a la marca de tiempo actual.

In [ ]:
t = time.time()  # Obtiene el tiempo actual en segundos.

export_path_keras = "./{}.h5".format(int(t))  # Crea un nombre de archivo para el modelo exportado en formato h5.
print(export_path_keras)  # Imprime la ruta del archivo donde se guardará el modelo en formato h5.

model.save(export_path_keras)  # Guarda el modelo en el archivo especificado en formato h5.


In [ ]:
!ls

Posteriormente podrá recrear el mismo modelo a partir de este archivo, incluso si ya no tiene acceso al código que creó el modelo.

Este archivo incluye:

- La arquitectura del modelo.
- Los valores de peso del modelo (que se aprendieron durante el entrenamiento)
- La configuración de entrenamiento del modelo (lo que pasaste a "compilar"), si corresponde.
- El optimizador y su estado, si lo hubiera (esto le permite reiniciar el entrenamiento donde lo dejó)

# Parte 4: Cargar el modelo `.h5` de Keras

Ahora cargaremos el modelo que acabamos de guardar en un nuevo modelo llamado "recargado". Necesitaremos proporcionar la ruta del archivo y el parámetro `custom_objects`. Este parámetro le dice a Keras cómo cargar `hub.KerasLayer` desde `feature_extractor` que usamos para el aprendizaje por transferencia.

In [ ]:
reloaded = tf.keras.models.load_model(
    export_path_keras,
    # Carga el modelo desde el archivo especificado en 'export_path_keras'.

    custom_objects={'KerasLayer': hub.KerasLayer}
    # Especifica cómo cargar la capa 'hub.KerasLayer' del modelo, ya que es una capa personalizada utilizada en la red neuronal.
)

reloaded.summary()
# Muestra un resumen del modelo cargado, incluyendo la arquitectura de las capas,
#el número de parámetros entrenables y no entrenables, y la forma de salida de cada capa.


Podemos comprobar que el modelo recargado y el modelo anterior dan el mismo resultado.

In [ ]:
result_batch = model.predict(image_batch)
# Realiza predicciones en el lote de imágenes 'image_batch' utilizando el modelo original ('model').

reloaded_result_batch = reloaded.predict(image_batch)
# Realiza predicciones en el mismo lote de imágenes 'image_batch' utilizando el modelo recargado ('reloaded').


La diferencia en la producción debe ser cero:

In [ ]:
(abs(result_batch - reloaded_result_batch)).max()
# Calcula el valor absoluto de la diferencia entre los resultados predichos por el modelo original y el modelo recargado para cada elemento del lote.
# Luego, encuentra el valor máximo de estas diferencias.
# Esta expresión evalúa la máxima discrepancia entre las predicciones del modelo original y el modelo recargado en el conjunto de imágenes proporcionado.
# Un valor cercano a cero indica que las predicciones son prácticamente idénticas entre los dos modelos para las imágenes dadas.


Como podemos ver, el resultado es 0,0, lo que indica que ambos modelos hicieron las mismas predicciones en el mismo lote de imágenes.

# Sigue entrenando

Además de hacer predicciones, también podemos tomar nuestro modelo `reloaded` y seguir entrenándolo. Para hacer esto, puedes entrenar el `reloaded` como de costumbre, usando el método `.fit`.

In [ ]:
EPOCHS = 3  # Número de épocas para entrenar el modelo recargado ('reloaded').

history = reloaded.fit(
    train_batches,  # Utiliza el lote de entrenamiento para entrenar el modelo.
    epochs=EPOCHS,  # Número de épocas para entrenar el modelo recargado.
    validation_data=validation_batches  # Utiliza el lote de validación para evaluar el modelo después de cada época de entrenamiento.
)
# Entrena el modelo recargado durante el número especificado de épocas utilizando los lotes de entrenamiento y validación.
# El progreso del entrenamiento y las métricas se almacenan en el objeto 'history' para su posterior análisis y visualización.


# Parte 5: Exportar como modelo guardado


También puede exportar un modelo completo al formato TensorFlow SavedModel. SavedModel es un formato de serialización independiente para objetos de Tensorflow, compatible con el servicio TensorFlow, así como con implementaciones de TensorFlow distintas de Python. Un SavedModel contiene un programa TensorFlow completo, que incluye pesos y cálculos. No requiere la ejecución del código de creación del modelo original, lo que lo hace útil para compartir o implementar (con TFLite, TensorFlow.js, TensorFlow Serving o TFHub).

Los archivos SavedModel que se crearon contienen:

* Un punto de control de TensorFlow que contiene los pesos del modelo.
* Un prototipo de SavedModel que contiene el gráfico subyacente de Tensorflow. Se guardan gráficos separados para predicción (publicación), entrenamiento y evaluación. Si el modelo no se compiló antes, solo se exporta el gráfico de inferencia.
* La configuración de arquitectura del modelo, si está disponible.


Guardemos nuestro `model` original como TensorFlow SavedModel. Para hacer esto usaremos la función `tf.saved_model.save()`. Esta función toma el modelo que queremos guardar y la ruta a la carpeta donde queremos guardar nuestro modelo.

Esta función creará una carpeta donde encontrará una carpeta `assets`, una carpeta `variables` y el archivo `saved_model.pb`.

In [ ]:
t = time.time()  # Obtiene el tiempo actual en segundos.

export_path_sm = "./{}".format(int(t))  # Crea un nombre de carpeta para el modelo exportado.
print(export_path_sm)  # Imprime la ruta de la carpeta donde se guardarán los archivos del modelo.

tf.saved_model.save(model, export_path_sm)
# Guarda el modelo utilizando el formato SavedModel en la carpeta especificada en 'export_path_sm'.


In [ ]:
!ls {export_path_sm}
# Utiliza el comando 'ls' para listar los archivos y directorios dentro de la carpeta especificada por 'export_path_sm'.



# Parte 6: Cargar modelo guardado

Ahora, carguemos nuestro SavedModel y usémoslo para hacer predicciones. Usamos la función `tf.saved_model.load()` para cargar nuestros SavedModels. El objeto devuelto por `tf.saved_model.load` es 100% independiente del código que lo creó.

In [ ]:
reloaded_sm = tf.saved_model.load(export_path_sm)
# Carga el modelo SavedModel desde la carpeta especificada en 'export_path_sm' utilizando la función 'tf.saved_model.load()'.

# Después de cargar el modelo SavedModel, el modelo reloaded_sm está listo para ser utilizado para la inferencia y otras operaciones de TensorFlow.
# Este paso es crucial para cargar el modelo guardado y poder usarlo en aplicaciones y servicios en producción.


Ahora, usemos `reloaded_sm` (Reloaded SavedModel) para hacer predicciones en un lote de imágenes.

In [ ]:
reload_sm_result_batch = reloaded_sm(image_batch, training=False).numpy()
# Utiliza el modelo SavedModel ('reloaded_sm') para realizar inferencias en el lote de imágenes 'image_batch'.
# La opción 'training=False' indica que el modelo se utiliza en modo de inferencia, no en modo de entrenamiento.

# La función devuelve un array NumPy que contiene las predicciones del modelo SavedModel en el conjunto de imágenes dado.
# Estas predicciones se almacenan en la variable 'reload_sm_result_batch' para su posterior análisis o visualización.


Podemos comprobar que el SavedModel recargado y el modelo anterior dan el mismo resultado.

In [ ]:
(abs(result_batch - reload_sm_result_batch)).max()
# Calcula el valor absoluto de la diferencia entre los resultados predichos por el modelo original y el modelo SavedModel para cada elemento del lote.
# Luego, encuentra el valor máximo de estas diferencias.

# Esta expresión evalúa la máxima discrepancia entre las predicciones del modelo original y el modelo SavedModel en el conjunto de imágenes proporcionado.
# Un valor cercano a cero indica que las predicciones son prácticamente idénticas entre los dos modelos para las imágenes dadas.



Como podemos ver, el resultado es 0,0, lo que indica que ambos modelos hicieron las mismas predicciones en el mismo lote de imágenes.

# Parte 7: Carga del modelo guardado como modelo de Keras

El objeto devuelto por `tf.saved_model.load` no es un objeto Keras (es decir, no tiene los métodos `.fit`, `.predict`, `.summary`, etc.). Por lo tanto, no puedes simplemente tomar tu modelo `reloaded_sm` y seguir entrenándolo ejecutando `.fit`. Para poder recuperar un modelo keras completo desde el formato Tensorflow SavedModel debemos usar la función `tf.keras.models.load_model`. Esta función funcionará igual que antes, excepto que ahora pasamos la ruta a la carpeta que contiene nuestro SavedModel.

In [ ]:
t = time.time()  # Obtiene el tiempo actual en segundos.

export_path_sm = "./{}".format(int(t))  # Crea un nombre de carpeta para el modelo exportado.
print(export_path_sm)  # Imprime la ruta de la carpeta donde se guardarán los archivos del modelo.

tf.saved_model.save(model, export_path_sm)
# Guarda el modelo utilizando el formato SavedModel en la carpeta especificada en 'export_path_sm'.


In [ ]:
reload_sm_keras = tf.keras.models.load_model(
    export_path_sm,
    custom_objects={'KerasLayer': hub.KerasLayer}
)
# Carga el modelo SavedModel desde la carpeta especificada en 'export_path_sm' utilizando 'tf.keras.models.load_model()'.
# También se especifica cómo cargar la capa personalizada 'hub.KerasLayer'.

reload_sm_keras.summary()
# Muestra un resumen del modelo cargado ('reload_sm_keras'), incluyendo la arquitectura de las capas,
#el número de parámetros entrenables y no entrenables, y la forma de salida de cada capa.


Ahora, usemos `reloaded_sm)keras` (modelo Keras recargado de nuestro SavedModel) para hacer predicciones en un lote de imágenes.

In [ ]:
result_batch = model.predict(image_batch)
# Utiliza el modelo original ('model') para realizar predicciones en el lote de imágenes 'image_batch'.

reload_sm_keras_result_batch = reload_sm_keras.predict(image_batch)
# Utiliza el modelo cargado desde el SavedModel ('reload_sm_keras') para hacer predicciones en el mismo lote de imágenes 'image_batch'.

# Estas líneas de código realizan inferencias en el conjunto de imágenes utilizando el modelo original y el modelo cargado desde el SavedModel respectivamente.
# Los resultados obtenidos se pueden utilizar para comparar las predicciones entre los dos modelos y verificar la consistencia en sus resultados.


Podemos comprobar que el modelo Keras recargado y el modelo anterior dan el mismo resultado.

In [ ]:
(abs(result_batch - reload_sm_keras_result_batch)).max()
# Calcula el valor absoluto de la diferencia entre las predicciones del modelo original y las del modelo cargado desde el SavedModel para cada elemento del lote.
# Luego, encuentra el valor máximo de estas diferencias.

# Esta expresión evalúa la máxima discrepancia entre las predicciones del modelo original y las del modelo cargado desde el SavedModel en el conjunto de imágenes proporcionado.
# Un valor cercano a cero indica que las predicciones son prácticamente idénticas entre los dos modelos para las imágenes dadas.
# Al comparar estas diferencias máximas, se verifica la consistencia entre las predicciones del modelo original y las del modelo cargado desde el SavedModel.


# Parte 8: Descarga tu modelo

Puede descargar SavedModel a su disco local creando un archivo zip. Usaremos la opción `-r` (recursiva) para comprimir todas las subcarpetas.

In [ ]:
!zip -r model.zip {export_path_sm}
# Utiliza el comando 'zip' para comprimir y empaquetar los archivos y directorios dentro de la carpeta especificada por 'export_path_sm'.

# Al ejecutar este comando en un entorno de línea de comandos, se creará un archivo ZIP llamado 'model.zip' que contiene los archivos y subcarpetas del modelo SavedModel.
# Esto facilita la transferencia y el almacenamiento del modelo en una forma compacta y comprimida, ideal para compartirlo o realizar copias de seguridad.
# El archivo ZIP resultante se puede utilizar para cargar el modelo SavedModel en otro entorno o sistema sin necesidad de enviar cada archivo individualmente.


El archivo zip se guarda en el directorio de trabajo actual. Puede ver cuál es el directorio de trabajo actual ejecutando:

In [ ]:
!ls
# Utiliza el comando 'ls' para listar los archivos y directorios en el directorio actual.


Una vez que el archivo esté comprimido, puede descargarlo a su disco local.

In [ ]:
try:
    from google.colab import files
    files.download('./model.zip')
except ImportError:
    pass
# Intenta descargar el archivo 'model.zip' utilizando la función 'files.download()' de Google Colab.

# En el entorno de Google Colab, esta función permite a los usuarios descargar archivos directamente desde el entorno Colab a su sistema local.
# Si la importación de 'files' y la función 'download()' tienen éxito, el archivo ZIP del modelo será descargado al sistema local del usuario.
# Si se produce un ImportError, lo cual puede suceder fuera del entorno de Google Colab, el código continúa sin hacer nada, ya que la función de descarga no está disponible en ese contexto.


El comando `files.download` buscará archivos en su directorio de trabajo actual. Si el archivo que desea descargar está en un directorio distinto al directorio de trabajo actual, debe incluir la ruta al directorio donde se encuentra el archivo.